In [1]:
import numpy as np
from scipy import stats
from mcc import mcc

In [2]:
y = 1.0 * np.random.choice(2, size = 200)
mvn = stats.multivariate_normal.rvs
X = np.array([mvn([1, 1] if heads else [4, 4]) for heads in y])

In [5]:
import lr
import newton
import bfgs
import irls
import logistic

logistic.minimize = newton.minimize

clf = logistic

In [6]:
model = clf.fit(X, y)
y_hat = clf.predict(model, X)
print np.mean(y == y_hat)
print mcc(y, y_hat)

0.535
0.202093008683
